# Clockodo API Testing

This notebook tests the Clockodo client functions to debug API interactions.

## Setup

Make sure you have a .env file in the project root with your Clockodo credentials.

## Usage

You can now run all cells sequentially! The notebook automatically:
- Fetches active (non-archived) services dynamically in each test
- Uses ISO 8601 datetime format for time entries
- Handles API requirements correctly

In [ ]:
import os
import sys
from pathlib import Path

# Add the src directory to Python path
notebook_dir = Path.cwd()
src_dir = notebook_dir.parent / 'src'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

print(f"Added {src_dir} to Python path")
print(f"Current working directory: {notebook_dir}")

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env from project root (parent directory of manual-test)
env_path = notebook_dir.parent / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print(f"✓ Loaded environment variables from {env_path}")
else:
    print(f"⚠️  Warning: .env file not found at {env_path}")
    print("Please create a .env file with CLOCKODO_API_USER and CLOCKODO_API_KEY")

## Import Clockodo Client

In [ ]:
from clockodo_mcp.client import ClockodoClient

# Create client from environment variables
client = ClockodoClient.from_env()

print(f"Client created:")
print(f"  API User: {client.api_user}")
print(f"  Base URL: {client.base_url}")
print(f"  User Agent: {client.user_agent}")

## Test 1: List Users

In [ ]:
import json

try:
    users_response = client.list_users()
    print("✓ Successfully retrieved users")
    print(f"\nRaw response:")
    print(json.dumps(users_response, indent=2))
    
    # Parse users if available
    if 'users' in users_response:
        users = users_response['users']
        print(f"\n📊 Found {len(users)} user(s)")
        for user in users:
            print(f"  - ID: {user.get('id')}, Name: {user.get('name')}, Email: {user.get('email')}")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 1a: List Customers

In [ ]:
try:
    customers_response = client.list_customers()
    print("✓ Successfully retrieved customers")
    
    if 'customers' in customers_response and len(customers_response['customers']) > 0:
        customers = customers_response['customers']
        print(f"\n📊 Found {len(customers)} customer(s)")
        # Store the first one for later tests
        first_customer_id = customers[0]['id']
        print(f"  - First Customer: {customers[0].get('name')} (ID: {first_customer_id})")
    else:
        print("No customers found")
        first_customer_id = None
except Exception as e:
    print(f"❌ Error: {e}")

# Helper function to get active service ID
def get_first_active_service_id():
    """Get the first active (non-archived) service ID."""
    try:
        services_response = client.list_services()
        if 'services' in services_response:
            services = services_response['services']
            # Filter for active services only
            active_services = [s for s in services if s.get('active') is True]
            if active_services:
                return active_services[0]['id']
        return None
    except Exception:
        return None


## Test 1b: List Services

This cell displays all services (active and archived). Later test cells automatically use the first active service.

In [ ]:
try:
    services_response = client.list_services()
    print("✓ Successfully retrieved services")
    
    if 'services' in services_response and len(services_response['services']) > 0:
        services = services_response['services']
        print(f"\n📊 Found {len(services)} service(s)")
        
        # Filter for active services (active == True)
        active_services = [s for s in services if s.get('active') is True]
        print(f"  Active services: {len(active_services)}")
        
        # Show all services with their status
        print("\nAll services:")
        for service in services:
            status = "ACTIVE" if service.get('active') is True else "ARCHIVED"
            print(f"  - {service.get('name')} (ID: {service['id']}) - {status}")
        
        # Display which service will be used
        if active_services:
            print(f"\n✓ First active service that will be used in tests:")
            print(f"  {active_services[0].get('name')} (ID: {active_services[0]['id']})")
        else:
            print("\n⚠️ No active services found!")
    else:
        print("No services found")
except Exception as e:
    print(f"❌ Error: {e}")

## Test 2: Get User Reports

This is the main test for the debug tool. We'll test fetching user reports for a specific year.

In [ ]:
# Test for current year
from datetime import datetime

current_year = datetime.now().year
print(f"Testing user reports for year: {current_year}")

try:
    reports_response = client.get_user_reports(year=current_year)
    print("✓ Successfully retrieved user reports")
    print(f"\nRaw response:")
    print(json.dumps(reports_response, indent=2))
    
    # Show response structure
    print(f"\n📊 Response keys: {list(reports_response.keys())}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 3: Get User Reports with Different Type Levels

The Clockodo API supports different type levels for aggregation.

In [ ]:
# Test different type levels (0, 1, 2, 3)
for type_level in [0, 1, 2, 3]:
    print(f"\n{'='*60}")
    print(f"Testing type_level={type_level}")
    print(f"{'='*60}")
    
    try:
        reports = client.get_user_reports(year=current_year, type_level=type_level)
        print(f"✓ Success")
        print(f"Response keys: {list(reports.keys())}")
        
        # Show a sample of the data
        print(f"\nSample data:")
        print(json.dumps(reports, indent=2)[:1000] + "..." if len(json.dumps(reports)) > 1000 else json.dumps(reports, indent=2))
        
    except Exception as e:
        print(f"❌ Error: {e}")

## Test 4: Get User Reports for Specific User

If you have multiple users, test fetching reports for a specific user.

In [ ]:
# First get the user ID from the users list
try:
    users_response = client.list_users()
    if 'users' in users_response and len(users_response['users']) > 0:
        user_id = users_response['users'][0]['id']
        print(f"Testing reports for user_id: {user_id}")
        
        reports = client.get_user_reports(year=current_year, user_id=user_id)
        print("✓ Successfully retrieved user-specific reports")
        print(f"\nResponse:")
        print(json.dumps(reports, indent=2))
    else:
        print("No users found to test with")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 5: Test Debug Tool Function

Test the actual debug tool function from the module.

In [ ]:
from clockodo_mcp.tools.debug_tools import get_raw_user_reports

print(f"Testing debug tool for year {current_year}")

try:
    result = get_raw_user_reports(year=current_year)
    print("✓ Debug tool executed successfully")
    print(f"\nResult:")
    print(json.dumps(result, indent=2))
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 6: Inspect HTTP Headers

Check what headers are being sent with requests.

In [ ]:
print("Headers being sent with requests:")
headers = client.default_headers
print(json.dumps(headers, indent=2))

## Test 7: Test Different Years

Test fetching reports for different years to see what data is available.

In [ ]:
# Test last 3 years
test_years = [current_year - 2, current_year - 1, current_year]

for year in test_years:
    print(f"\n{'='*60}")
    print(f"Year: {year}")
    print(f"{'='*60}")
    
    try:
        reports = client.get_user_reports(year=year)
        print(f"✓ Success")
        
        # Try to extract some summary info
        if 'userreports' in reports:
            print(f"  Found 'userreports' key")
            print(f"  Data type: {type(reports['userreports'])}")
            if isinstance(reports['userreports'], list):
                print(f"  Number of entries: {len(reports['userreports'])}")
        else:
            print(f"  Response keys: {list(reports.keys())}")
            
    except Exception as e:
        print(f"❌ Error: {e}")

## Test 8: User Service Identification

Test the UserService and how it identifies the current user ID.

In [ ]:
from clockodo_mcp.services.user_service import UserService

user_service = UserService(client)

try:
    user_id = user_service.get_current_user_id()
    print(f"✓ Current user ID: {user_id}")
    print(f"  Identified by email: {client.api_user}")
except Exception as e:
    print(f"❌ Error: {e}")

## Test 9: Clock Operations

Test getting, starting, and stopping the clock.

In [ ]:
# Initialize clock_entry_id for cleanup
clock_entry_id = None

try:
    # Get active service ID dynamically
    first_service_id = get_first_active_service_id()
    
    # 1. Get current clock
    clock_resp = user_service.get_my_clock()
    print("✓ Successfully retrieved current clock status")
    print(json.dumps(clock_resp, indent=2))
    
    # 2. Check if clock is running
    if clock_resp.get('running'):
        print("\n⚠️ Clock is currently running. Stopping it first...")
        user_service.stop_my_clock()
        print("✓ Clock stopped")
    else:
        print("\n✓ Clock is NOT running.")
        
    # Note: Starting/Stopping the clock will create real data in Clockodo!
    print(f"\nTesting clock start (WILL CREATE REAL DATA)...")
    print(f"  Customer ID: {first_customer_id}")
    print(f"  Service ID: {first_service_id}")
    
    if first_customer_id and first_service_id:
        result = user_service.start_my_clock(
            customers_id=first_customer_id, 
            services_id=first_service_id
        )
        print("✓ Successfully started clock")
        print(json.dumps(result, indent=2))
        
        # Capture the clock entry ID for cleanup
        if result.get('running') and result['running'].get('id'):
            clock_entry_id = result['running']['id']
            print(f"\n✓ Captured clock entry ID: {clock_entry_id}")
        
        # Stop it immediately
        print("\n  Stopping clock immediately...")
        stop_result = user_service.stop_my_clock()
        print("✓ Clock stopped")
        
        # The stopped entry ID is what we need to delete
        if stop_result.get('stopped') and stop_result['stopped'].get('id'):
            clock_entry_id = stop_result['stopped']['id']
            print(f"✓ Clock entry ID to cleanup: {clock_entry_id}")
    else:
        print("⚠️ Skipping clock start: No customer or active service available")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 10: Absences (Vacation)

Test listing absences and adding a vacation.

In [ ]:
# Initialize last_absence_id at the start
last_absence_id = None

try:
    # List absences for current year
    absences = client.list_absences(year=current_year)
    print(f"✓ Successfully retrieved absences for {current_year}")
    print(f"  Found {len(absences.get('absences', []))} absence(s)")
    
    # Show first absence if any
    if absences.get('absences'):
        print("\nSample absence:")
        print(json.dumps(absences['absences'][0], indent=2))
        
    # Note: add_my_vacation will create a real absence!
    # The created absence will be auto-approved (status: 1)
    # To delete it, you need to decline it first or delete it via the UI
    print("\nTesting add vacation (WILL CREATE REAL DATA - commented out by default)...")
    # Uncomment to test:
    absence_result = user_service.add_my_vacation(date_since="2025-12-24", date_until="2025-12-26")
    print(json.dumps(absence_result, indent=2))
    if "absence" in absence_result:
        last_absence_id = absence_result["absence"]["id"]
        print(f"\nCaptured absence ID for cleanup: {last_absence_id}")
    
except Exception as e:
    print(f"❌ Error: {e}")

## Test 11: Time Entries

Test listing and adding time entries.

In [ ]:
# Initialize last_entry_id at the start
last_entry_id = None

try:
    # Get active service ID dynamically
    first_service_id = get_first_active_service_id()
    
    # List entries for today - USE ISO 8601 FORMAT: YYYY-MM-DDTHH:MM:SSZ
    today = datetime.now().strftime('%Y-%m-%d')
    time_since = f"{today}T00:00:00Z"
    time_until = f"{today}T23:59:59Z"
    
    entries = user_service.get_my_entries(time_since=time_since, time_until=time_until)
    print(f"✓ Successfully retrieved entries for {today}")
    print(f"  Found {len(entries.get('entries', []))} entry(s)")
    
    if entries.get('entries'):
        print("\nSample entry:")
        print(json.dumps(entries['entries'][0], indent=2))
        
    # Note: add_my_entry will create a real time entry!
    print("\nTesting add entry (WILL CREATE REAL DATA)...")
    print(f"  Customer ID: {first_customer_id}")
    print(f"  Service ID: {first_service_id}")
    print(f"  Time range: {today}T09:00:00Z to {today}T10:00:00Z")
    
    if first_customer_id and first_service_id:
        entry_result = user_service.add_my_entry(
            customers_id=first_customer_id, 
            services_id=first_service_id,
            billable=1,
            time_since=f"{today}T09:00:00Z",
            time_until=f"{today}T10:00:00Z",
            text="Manual test entry from notebook"
        )
        print("✓ Successfully added entry")
        print(json.dumps(entry_result, indent=2))
        
        if "entry" in entry_result:
            last_entry_id = entry_result["entry"]["id"]
            print(f"\n✓ Captured entry ID for cleanup: {last_entry_id}")
    else:
        print("⚠️ Skipping add entry: No customer or active service available")

except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 12: Cleanup

Delete the time entry created in the previous test.

In [ ]:
## Test 12: Cleanup (Delete created entries and absences)
try:
    # Cleanup Clock Entry (from Test 9)
    if 'clock_entry_id' in globals() and clock_entry_id:
        print(f"Cleaning up clock entry {clock_entry_id}...")
        delete_result = user_service.delete_my_entry(clock_entry_id)
        print("✓ Successfully deleted clock entry")
    else:
        print("No clock entry ID to cleanup")
    
    # Cleanup Time Entry (from Test 11)
    if last_entry_id:
        print(f"\nCleaning up time entry {last_entry_id}...")
        delete_result = user_service.delete_my_entry(last_entry_id)
        print("✓ Successfully deleted time entry")
    else:
        print("\nNo time entry ID to cleanup")
    
    # Cleanup Absence - Try to cancel and then delete
    # Status workflow: 1 (approved) → 3 (approval cancelled) → delete
    if last_absence_id:
        print(f"\nCleaning up absence {last_absence_id}...")
        print("  Step 1: Cancelling approved absence (status 1 → 3)...")
        try:
            cancel_result = user_service.cancel_my_vacation(last_absence_id)
            print("  ✓ Successfully cancelled absence (now status 3)")
            
            print("  Step 2: Deleting cancelled absence...")
            delete_result = user_service.delete_my_vacation(last_absence_id)
            print("✓ Successfully deleted absence")
        except Exception as absence_error:
            print(f"❌ Error: {absence_error}")
            print(f"   Note: Deletion requires admin rights for absence administration")
            print(f"   Please delete absence ID {last_absence_id} manually via Clockodo UI")
    else:
        print("\nNo absence ID to cleanup")
        
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    import traceback
    traceback.print_exc()